# Bright By Text: Deactivation Rates

## Setup and Data Import

In [3]:
# Widen notebook

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Import libraries necessary
import numpy as np
import sys
np.set_printoptions(threshold=sys.maxsize)
import pandas as pd
from time import time
import datetime
from IPython.display import display
import re
%matplotlib inline
import psycopg2
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
pd.options.display.float_format = '{:.3f}'.format

In [167]:
# Connect to PostgeSQL

try:
    conn = psycopg2.connect(user = "pbiusr1",
                                  password = "RE_^V%Gj@EL6R!G",
                                  host = "app.brightbytext.org",
                                  port = "5432",
                                  database = "bbtapi")

except (Exception, psycopg2.Error) as error :
    print ("Error while connecting to PostgreSQL", error)

In [168]:
# Query to get subscriber info

sql = '''
    select
    s.id as subscriber_id,
    s.created_at,
    s.signedup_at,
    s.deactivated_at,
    s.carrier_name,
    z.code as subscriber_zip_code,
    z.city as subscriber_city,
    z.low_income,
    z.hpsa,
    s_state.name as subscriber_state,
    ss.name as subscriber_source,
    sst.name as subscriber_status,
    l.name as subscriber_language,
    sdr.name as subscriber_deactivation_method,
    c.children_count,
    c.dob_youngest_child,
    c.dob_oldest_child,
    t.default_offset as timezone_default_offset,
    t.iana_name as timezone_name
from public.v_subscribers as s
left join public.zip_codes as z 
on z.id = s.zip_id
left join public.states as s_state
on s_state.id = z.state_id
left join public.subscriber_sources as ss 
on ss.id = s.source_id
left join public.subscriber_statuses as sst 
on sst.id = s.status_id
left join public.languages as l
on l.id = s.language_id
left join public.subscriber_deactivation_reasons as sdr 
on sdr.id = s.deactivation_reason_id
left join 
(
    select subscriber_id, count(*) as children_count, max(date_of_birth) as dob_youngest_child, min(date_of_birth) as dob_oldest_child
    from public.children 
    group by subscriber_id
) c 
on s.id = c.subscriber_id
left join public.timezones t
on z.time_zone_id = t.id
'''
subscribers = pd.read_sql_query(sql, conn)
conn = None

In [169]:
subscribers

,subscriber_id,created_at,signedup_at,deactivated_at,carrier_name,subscriber_zip_code,subscriber_city,low_income,hpsa,subscriber_state,subscriber_source,subscriber_status,subscriber_language,subscriber_deactivation_method,children_count,dob_youngest_child,dob_oldest_child,timezone_default_offset,timezone_name
0,bb6f86cf-531b-454e-b5dd-f289b7f1fdbc,2018-10-01 22:01:46.917124,2018-10-01 22:01:46.917124,NaT,AT&T Wireless,27312,Pittsboro,False,False,North Carolina,Text SignUp,activated,English,None,1.000,2014-02-19,2014-02-19,-5.000,America/New_York
1,2008c3e2-07fb-4143-b1b9-267d4f83ea3c,2017-12-27 21:26:17.417336,2017-12-27 21:26:17.417336,NaT,AT&T Wireless,76137,Fort Worth,False,False,Texas,Text SignUp,activated,English,None,1.000,2017-05-19,2017-05-19,-6.000,America/Chicago
2,a7bdfa6e-ecdb-4993-8ae0-28edb365de66,2020-05-03 18:16:36.833859,2020-05-03 18:16:36.833859,NaT,Verizon Wireless,80621,Fort Lupton,True,True,Colorado,Web SignUp,activated,English,None,1.000,2020-05-01,2020-05-01,-7.000,America/Denver
3,a72ba682-d702-4381-a2b9-290ad5ac1910,2019-05-04 22:26:50.429979,2019-05-04 22:30:59.551576,NaT,Verizon Wireless,80903,Colorado Springs,True,True,Colorado,Text SignUp,activated,English,None,1.000,2018-11-29,2018-11-29,-7.000,America/Denver
4,bb66be5d-f2ae-449e-9e09-e191d0dcb43f,2017-05-02 16:34:10.730656,2019-12-14 19:37:16.195069,NaT,Google (Grand Central) BWI - Bandwidth.com - SVR,11212,Brooklyn,True,True,New York,Text SignUp,activated,English,None,2.000,2017-10-06,2016-05-04,-5.000,America/New_York
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122074,3e29a6e1-3778-41ca-8745-c146242f4cf1,2018-07-19 00:35:31.171563,2018-07-19 00:35:31.171563,NaT,"Sprint Spectrum, L.P.",78724,Austin,True,False,Texas,Text SignUp,activated,English,None,1.000,2018-03-28,2018-03-28,-6.000,America/Chicago
122075,bb4bb61b-096a-486d-be4d-bb635a0d76a6,2018-09-22 21:30:57.769983,2018-09-22 21:30:57.769983,NaT,Verizon Wireless,06854,Norwalk,False,False,Connecticut,Text SignUp,activated,English,None,1.000,2015-05-22,2015-05-22,-5.000,America/New_York
122076,261b80b2-734d-4256-9127-7add06ed6cf8,2019-02-13 17:03:10.161648,2019-02-13 17:03:10.161648,NaT,"Sprint Spectrum, L.P.",28269,Charlotte,False,False,North Carolina,Text SignUp,activated,English,None,1.000,2018-12-12,2018-12-12,-5.000,America/New_York
122077,3e2b537b-3c83-4133-9385-4b2a6eba7741,2017-05-12 18:00:01.021358,2017-05-12 18:00:01.021358,NaT,Verizon Wireless,27577,Smithfield,True,True,North Carolina,Text SignUp,activated,English,None,1.000,2016-04-11,2016-04-11,-5.000,America/New_York


In [170]:
subscribers['dob_youngest_child'] = pd.to_datetime(subscribers['dob_youngest_child'], errors = 'coerce') # Change DOB of youngest child to datetime format
subscribers['dob_oldest_child'] = pd.to_datetime(subscribers['dob_oldest_child'], errors = 'coerce') # Change DOB of oldest child to datetime format
subscribers['created_at'] = pd.to_datetime(subscribers['created_at']) # Change created at timestamp to datetime format

In [171]:
subscribers['status'] = np.where((subscribers['subscriber_status'] == 'activated') | (subscribers['subscriber_status'] == 'pending'), 'activated', subscribers['subscriber_deactivation_method'])
subscribers['status'] = subscribers['status'].fillna(value = 'other deactivated')
subscribers['spanish'] = np.where(subscribers['subscriber_language'] == 'Spanish', 1, 0)
subscribers['hpsa'] = np.where(subscribers['hpsa'] == True, 1, 0)
subscribers['low_income'] = np.where(subscribers['low_income'] == True, 1, 0)

In [87]:
subscribers.groupby(['subscriber_status', 'hpsa'])['subscriber_id'].count()

subscriber_status  hpsa 
activated          False    41134
                   True     35934
deactivated        False    11449
                   True     10672
pending            False       11
                   True         6
Name: subscriber_id, dtype: int64

## By Language

In [78]:
pd.DataFrame(subscribers.groupby(['subscriber_language', 'status'])['subscriber_id'].count()).reset_index()

,subscriber_language,status,subscriber_id
0,English,Age Out,456
1,English,Deactivated by Admin,18
2,English,Invalid Phone,7
3,English,STOP,25655
4,English,Signup not Completed,12638
5,English,activated,77458
6,English,other deactivated,118
7,Spanish,Age Out,25
8,Spanish,Deactivated by Admin,2
9,Spanish,STOP,790


In [80]:
# Export to Excel

pd.DataFrame(subscribers.groupby(['subscriber_language', 'status'])['subscriber_id'].count()).reset_index().to_csv(r'/Users/lisafan/Desktop/BBT/deactivation_by_language.csv', index=False)  

## By HPSA

In [83]:
pd.DataFrame(subscribers.groupby(['hpsa', 'status'])['subscriber_id'].count()).reset_index()

,hpsa,status,subscriber_id
0,False,Age Out,221
1,False,Deactivated by Admin,7
2,False,Invalid Phone,3
3,False,STOP,10938
4,False,Signup not Completed,225
5,False,activated,41145
6,False,other deactivated,55
7,True,Age Out,224
8,True,Deactivated by Admin,10
9,True,Invalid Phone,4


In [81]:
# Export to Excel

pd.DataFrame(subscribers.groupby(['hpsa', 'status'])['subscriber_id'].count()).reset_index().to_csv(r'/Users/lisafan/Desktop/BBT/deactivation_by_hpsa.csv', index=False)  

## By Low Income

In [84]:
pd.DataFrame(subscribers.groupby(['low_income', 'status'])['subscriber_id'].count()).reset_index()

,low_income,status,subscriber_id
0,False,Age Out,167
1,False,Deactivated by Admin,7
2,False,Invalid Phone,3
3,False,STOP,9576
4,False,Signup not Completed,209
5,False,activated,33333
6,False,other deactivated,42
7,True,Age Out,278
8,True,Deactivated by Admin,10
9,True,Invalid Phone,4


In [82]:
# Export to Excel

pd.DataFrame(subscribers.groupby(['low_income', 'status'])['subscriber_id'].count()).reset_index().to_csv(r'/Users/lisafan/Desktop/BBT/deactivation_by_low_income.csv', index=False)  

# Bright By Text: Updated Subscriber Regression

In [137]:
# Load intermediate dataset

messages = pd.read_csv(r'/Users/lisafan/Desktop/BBT/bbt_messages_staging.csv', low_memory = False)

In [146]:
# Data cleaning

messages['outbound_message_created_at'] = pd.to_datetime(messages['outbound_message_created_at']) # Change message timestamp to datetime format
messages['signedup_at'] = pd.to_datetime(messages['signedup_at']) # Change signup timestamp to datetime format
messages['deactivated_at'] = pd.to_datetime(messages['deactivated_at']) # Change deactivation timestamp to datetime format
messages['created_at'] = pd.to_datetime(messages['created_at']) # Change created at timestamp to datetime format
messages['subscriber_zip_code'] = messages['subscriber_zip_code'].fillna(-1) # Fill NAs in order to convert to integers
messages['subscriber_zip_code'] = messages['subscriber_zip_code'].astype(int).astype(str).str.zfill(5) # Pad zip codes with 0s
messages['subscriber_zip_code'] = messages['subscriber_zip_code'].replace('000-1', np.nan) # Fill NAs again

In [155]:
messages['outbound_message_created_at'].max()

Timestamp('2020-04-03 07:27:01.294352')

In [156]:
messages['days_since_deactivated'] = messages['deactivated_at'] - messages['created_at']
messages['days_since_signup'] = pd.Timestamp('2020-04-03T08') - messages['created_at']
messages['subscriber_days'] = messages[['days_since_deactivated', 'days_since_signup']].min(axis = 1)
messages['subscriber_days'] = messages['subscriber_days'] / datetime.timedelta(days = 1)
messages['subscriber_weeks'] = messages['subscriber_days'] / 7
messages

,message_id,outbound_message_created_at,outbound_message_body,outbound_message_partner_id,message_type,subscriber_id,created_at,signedup_at,deactivated_at,carrier_name,subscriber_zip_code,subscriber_city,timezone_default_offset,timezone_name,subscriber_state,subscriber_source,subscriber_status,subscriber_language,subscriber_deactivation_method,partner_name,partner_created_date,partner_is_active,partner_state,children_count,dob_oldest_child,dob_youngest_child,scheduled_message_id,year,timezone,timezone_default_offset_adj,outbound_message_created_at_adj,deactivated_at_adj,created_at_adj,days_since_deactivated,days_since_signup,subscriber_days,subscriber_weeks
0,6e92ae11-f983-479f-8e9c-2b689368954d,2017-05-03 15:11:30.588685,Invalid zipcode. Kindly check your zipcode and...,00000000-0000-0000-0000-000000000002,signup,53e4abdd-308a-4cfc-a6fc-410c89a28d2d,2017-05-03 14:30:08.345377,2017-05-03 14:30:08.345377,NaT,AT&T Wireless,11096,Inwood,-5.000,America/New_York,New York,Text SignUp,activated,English,NaN,"WNET - New York, NY",2018-03-25 21:32:21.001262,True,New York,1.000,2015-06-06,2015-06-06,NaN,2017,Eastern,-1 days +20:00:00,2017-05-03 11:11:30.588685,NaT,2017-05-03 10:30:08.345377,NaT,1065 days 17:29:51.654623,1065.729,152.247
1,57c0ec0e-f812-4b45-a33a-213f4663f1c2,2017-05-09 16:34:15.603110,Is this zipcode correct? If yes reply with 'Y...,00000000-0000-0000-0000-000000000002,signup,f328ee46-398c-422f-aa55-4816caeae06e,2017-05-09 16:31:26.230154,2017-05-09 16:31:26.230154,NaT,"T-Mobile USA, Inc.",11219,Brooklyn,-5.000,America/New_York,New York,Text SignUp,activated,English,NaN,"WNET - New York, NY",2018-03-25 21:32:21.001262,True,New York,1.000,2014-06-23,2014-06-23,NaN,2017,Eastern,-1 days +20:00:00,2017-05-09 12:34:15.603110,NaT,2017-05-09 12:31:26.230154,NaT,1059 days 15:28:33.769846,1059.645,151.378
2,17d99d4d-137a-4317-b4d8-a63a4690fc93,2017-05-04 18:40:41.904659,"If you care for a young child, Bright by Text ...",f92494a8-bcce-473f-a56e-0ae2515c827e,signup,99915cfb-856b-4d59-8b94-0bd26d1342c4,2017-05-04 18:39:17.165132,2017-05-04 18:39:17.165132,NaT,"T-Mobile USA, Inc.",78251,San Antonio,-6.000,America/Chicago,Texas,Text SignUp,activated,English,NaN,KLRN - San Antonio,2018-08-28 20:58:04.700242,True,Texas,2.000,2014-09-02,2016-09-03,NaN,2017,Central,-1 days +19:00:00,2017-05-04 13:40:41.904659,NaT,2017-05-04 13:39:17.165132,NaT,1064 days 13:20:42.834868,1064.556,152.079
3,ffb5de23-d828-4e3a-becf-8b10b80cefba,2017-05-04 18:45:12.364195,Unrecognized date of birth format. Please res...,f92494a8-bcce-473f-a56e-0ae2515c827e,signup,99915cfb-856b-4d59-8b94-0bd26d1342c4,2017-05-04 18:39:17.165132,2017-05-04 18:39:17.165132,NaT,"T-Mobile USA, Inc.",78251,San Antonio,-6.000,America/Chicago,Texas,Text SignUp,activated,English,NaN,KLRN - San Antonio,2018-08-28 20:58:04.700242,True,Texas,2.000,2014-09-02,2016-09-03,NaN,2017,Central,-1 days +19:00:00,2017-05-04 13:45:12.364195,NaT,2017-05-04 13:39:17.165132,NaT,1064 days 13:20:42.834868,1064.556,152.079
4,bc56c5c2-c5b2-4a95-bfd5-a284661e7052,2017-05-04 18:39:17.856495,KLRN welcomes you to Bright by Text! Expect 2 ...,f92494a8-bcce-473f-a56e-0ae2515c827e,signup,99915cfb-856b-4d59-8b94-0bd26d1342c4,2017-05-04 18:39:17.165132,2017-05-04 18:39:17.165132,NaT,"T-Mobile USA, Inc.",78251,San Antonio,-6.000,America/Chicago,Texas,Text SignUp,activated,English,NaN,KLRN - San Antonio,2018-08-28 20:58:04.700242,True,Texas,2.000,2014-09-02,2016-09-03,NaN,2017,Central,-1 days +19:00:00,2017-05-04 13:39:17.856495,NaT,2017-05-04 13:39:17.165132,NaT,1064 days 13:20:42.834868,1064.556,152.079
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17349497,f6dfbbbd-1604-4ca8-b1c4-d75709a8641f,2020-04-02 22:05:25.285664,Are you worried about paying rent or utilities...,b6a20a01-a55e-481c-9878-7c0ce616374a,scheduled,ef99f698-0fce-46af-8a36-d6fb1e3019f3,2020-01-09 23:50:28.983049,2020-01-09 23:50:28.983049,NaT,US

In [157]:
# Messages per subscriber

messages_per_subscriber = messages.groupby('subscriber_id').agg({'message_id':'count', 'subscriber_weeks': 'max'}).reset_index()
messages_per_subscriber.columns = ['subscriber_id', 'message_count', 'subscriber_weeks']
messages_per_subscriber['messages_per_subscriber_per_week'] = messages_per_subscriber['message_count'] / messages_per_subscriber['subscriber_weeks']
messages_per_subscriber

,subscriber_id,message_count,subscriber_weeks,messages_per_subscriber_per_week
0,000018d6-c7fe-4b3c-8bf7-8502e4ae2d9b,293,87.167,3.361
1,00005a8a-477e-46c6-82f9-e4678760599c,48,19.982,2.402
2,0000661e-1c9c-4dde-a9f6-b2a8291142fd,123,34.367,3.579
3,00007706-12cc-48e8-a732-649de7ec8b50,141,24.247,5.815
4,000128d7-701e-4919-ad93-fd2ef3a5f69d,224,56.927,3.935
...,...,...,...,...
111601,fffc68c4-e8b7-4060-bce9-e6b6ca74b00c,303,105.509,2.872
111602,fffd402d-4bbd-4322-8389-a159f1dfa038,40,6.251,6.398
111603,fffdbcab-63d2-4158-9743-bbb138036518,6,0.318,18.848
111604,fffde26d-0880-4ca0-a67e-46dcd77f4dab,468,145.540,3.216


## Add in number of children and age of children as variables

In [173]:
subscribers['age_oldest_child_at_signup'] = (subscribers['created_at'] - subscribers['dob_oldest_child']) / pd.to_timedelta(1, unit='D')
subscribers['age_oldest_child_at_signup'] = subscribers['age_oldest_child_at_signup'] / 365.25

subscribers['age_youngest_child_at_signup'] = (subscribers['created_at'] - subscribers['dob_youngest_child']) / pd.to_timedelta(1, unit='D')
subscribers['age_youngest_child_at_signup'] = subscribers['age_youngest_child_at_signup'] / 365.25
subscribers

,subscriber_id,created_at,signedup_at,deactivated_at,carrier_name,subscriber_zip_code,subscriber_city,low_income,hpsa,subscriber_state,subscriber_source,subscriber_status,subscriber_language,subscriber_deactivation_method,children_count,dob_youngest_child,dob_oldest_child,timezone_default_offset,timezone_name,status,spanish,age_oldest_child_at_signup,age_youngest_child_at_signup
0,bb6f86cf-531b-454e-b5dd-f289b7f1fdbc,2018-10-01 22:01:46.917124,2018-10-01 22:01:46.917124,NaT,AT&T Wireless,27312,Pittsboro,0,0,North Carolina,Text SignUp,activated,English,None,1.000,2014-02-19,2014-02-19,-5.000,America/New_York,activated,0,4.616,4.616
1,2008c3e2-07fb-4143-b1b9-267d4f83ea3c,2017-12-27 21:26:17.417336,2017-12-27 21:26:17.417336,NaT,AT&T Wireless,76137,Fort Worth,0,0,Texas,Text SignUp,activated,English,None,1.000,2017-05-19,2017-05-19,-6.000,America/Chicago,activated,0,0.610,0.610
2,a7bdfa6e-ecdb-4993-8ae0-28edb365de66,2020-05-03 18:16:36.833859,2020-05-03 18:16:36.833859,NaT,Verizon Wireless,80621,Fort Lupton,1,1,Colorado,Web SignUp,activated,English,None,1.000,2020-05-01,2020-05-01,-7.000,America/Denver,activated,0,0.008,0.008
3,a72ba682-d702-4381-a2b9-290ad5ac1910,2019-05-04 22:26:50.429979,2019-05-04 22:30:59.551576,NaT,Verizon Wireless,80903,Colorado Springs,1,1,Colorado,Text SignUp,activated,English,None,1.000,2018-11-29,2018-11-29,-7.000,America/Denver,activated,0,0.430,0.430
4,bb66be5d-f2ae-449e-9e09-e191d0dcb43f,2017-05-02 16:34:10.730656,2019-12-14 19:37:16.195069,NaT,Google (Grand Central) BWI - Bandwidth.com - SVR,11212,Brooklyn,1,1,New York,Text SignUp,activated,English,None,2.000,2017-10-06,2016-05-04,-5.000,America/New_York,activated,0,0.996,-0.428
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122074,3e29a6e1-3778-41ca-8745-c146242f4cf1,2018-07-19 00:35:31.171563,2018-07-19 00:35:31.171563,NaT,"Sprint Spectrum, L.P.",78724,Austin,1,0,Texas,Text SignUp,activated,English,None,1.000,2018-03-28,2018-03-28,-6.000,America/Chicago,activated,0,0.309,0.309
122075,bb4bb61b-096a-486d-be4d-bb635a0d76a6,2018-09-22 21:30:57.769983,2018-09-22 21:30:57.769983,NaT,Verizon Wireless,06854,Norwalk,0,0,Connecticut,Text SignUp,activated,English,None,1.000,2015-05-22,2015-05-22,-5.000,America/New_York,activated,0,3.340,3.340
122076,261b80b2-734d-4256-9127-7add06ed6cf8,2019-02-13 17:03:10.161648,2019-02-13 17:03:10.161648,NaT,"Sprint Spectrum, L.P.",28269,Charlotte,0,0,North Carolina,Text SignUp,activated,English,None,1.000,2018-12-12,2018-12-12,-5.000,America/New_York,activated,0,0.174,0.174
122077,3e2b537b-3c83-4133-9385-4b2a6eba7741,2017-05-12 18:00:01.021358,2017-05-12 18:00:01.021358,NaT,Verizon Wireless,27577,Smithfield,1,1,North Carolina,Text SignUp,activated,English,None,1.000,2016-04-11,2016-04-11,-5.000,America/New_York,activated,0,1.086,1.086


## Analyze Correlations

In [177]:
subscribers_staging = subscribers[(subscribers['children_count'] > 0) & (subscribers['age_youngest_child_at_signup'] > -0.75) & (subscribers['age_youngest_child_at_signup'] <= 10)]
subscribers_staging = subscribers_staging[(subscribers_staging['status'] == 'activated') | (subscribers_staging['status'] == 'STOP')]
subscribers_staging['stop'] = np.where(subscribers_staging['status'] == 'STOP', 1, 0)
subscribers_staging = subscribers_staging[['subscriber_id', 'age_youngest_child_at_signup', 'age_oldest_child_at_signup', 'stop', 'children_count', 'spanish', 'hpsa', 'low_income']]
subscribers_staging

,subscriber_id,age_youngest_child_at_signup,age_oldest_child_at_signup,stop,children_count,spanish,hpsa,low_income
0,bb6f86cf-531b-454e-b5dd-f289b7f1fdbc,4.616,4.616,0,1.000,0,0,0
1,2008c3e2-07fb-4143-b1b9-267d4f83ea3c,0.610,0.610,0,1.000,0,0,0
2,a7bdfa6e-ecdb-4993-8ae0-28edb365de66,0.008,0.008,0,1.000,0,1,1
3,a72ba682-d702-4381-a2b9-290ad5ac1910,0.430,0.430,0,1.000,0,1,1
4,bb66be5d-f2ae-449e-9e09-e191d0dcb43f,-0.428,0.996,0,2.000,0,1,1
...,...,...,...,...,...,...,...,...
122073,2619ab21-16af-4f00-9ed8-52eefe951b71,1.603,1.603,0,1.000,0,1,1
122074,3e29a6e1-3778-41ca-8745-c146242f4cf1,0.309,0.309,0,1.000,0,0,1
122075,bb4bb61b-096a-486d-be4d-bb635a0d76a6,3.340,3.340,0,1.000,0,0,0
122076,261b80b2-734d-4256-9127-7add06ed6cf8,0.174,0.174,0,1.000,0,0,0


In [179]:
a = subscribers_staging.merge(messages_per_subscriber[messages_per_subscriber['subscriber_weeks'] >= 1], how = 'inner', left_on = 'subscriber_id', right_on = 'subscriber_id', left_index = True, right_index = False)
subscriber_reg = a.drop(columns = ['subscriber_id', 'message_count', 'subscriber_weeks'])
subscriber_reg.corr()

,age_youngest_child_at_signup,age_oldest_child_at_signup,stop,children_count,spanish,hpsa,low_income,messages_per_subscriber_per_week
age_youngest_child_at_signup,1.000,0.871,-0.021,-0.097,0.038,0.023,0.045,-0.402
age_oldest_child_at_signup,0.871,1.000,-0.014,0.325,0.042,0.035,0.054,-0.213
stop,-0.021,-0.014,1.000,0.012,-0.055,0.009,-0.024,-0.019
children_count,-0.097,0.325,0.012,1.000,0.016,0.034,0.028,0.377
spanish,0.038,0.042,-0.055,0.016,1.000,0.018,0.089,0.030
hpsa,0.023,0.035,0.009,0.034,0.018,1.000,0.449,-0.022
low_income,0.045,0.054,-0.024,0.028,0.089,0.449,1.000,0.028
messages_per_subscriber_per_week,-0.402,-0.213,-0.019,0.377,0.030,-0.022,0.028,1.000


HPSA and low income are highly correlated, so I will drop HPSA.

In [180]:
subscriber_reg.to_csv(r'/Users/lisafan/Desktop/BBT/new_subscriber_reg.csv', index=False)  

# Bright By Text: Survey Response Rates (Work in Progress)

In [130]:
# Connect to PostgeSQL

try:
    conn = psycopg2.connect(user = "pbiusr1",
                                  password = "RE_^V%Gj@EL6R!G",
                                  host = "app.brightbytext.org",
                                  port = "5432",
                                  database = "bbtapi")

except (Exception, psycopg2.Error) as error :
    print ("Error while connecting to PostgreSQL", error)

In [131]:
sql = '''
select *
from public.v_subscribers
'''
s = pd.read_sql_query(sql, conn)
conn = None

In [132]:
s

,id,program_id,partner_id,language_id,status_id,source_id,carrier_name,created_at,updated_at,unique_id,zip_id,survey_id,external_id,signedup_at,program_message_time,deactivated_at,deactivation_reason_id,signup_keyword_id
0,012dbd56-a5af-4683-853b-a29948e8f77d,00000000-0000-0000-0000-000000000001,d4d696af-37ff-427e-8035-2cb1bd630495,1,2,1,Verizon Wireless,2019-03-21 12:13:39.634156,2019-04-08 07:11:16.028449,bbt64e4f26a,13063.000,bbt2694cc59,71123,2019-03-21 12:13:39.634156,11:00:00,NaT,nan,00000000-0000-0000-0000-000000000003
1,bb6f86cf-531b-454e-b5dd-f289b7f1fdbc,00000000-0000-0000-0000-000000000001,97f6bf86-bbab-4e74-b3a9-3940bbc340a9,1,2,2,AT&T Wireless,2018-10-01 22:01:46.917124,2019-04-08 07:19:40.033166,bbtda59a75f,11336.000,bbtf98657fc,57657,2018-10-01 22:01:46.917124,11:00:00,NaT,nan,6c3e28fe-55bc-4623-a2be-dc91736f1285
2,012ed041-b331-47f7-93f6-dee9a52f8fc8,00000000-0000-0000-0000-000000000001,24bf3729-46b3-47fa-ba19-6786f208758b,1,2,2,AT&T Wireless,2018-02-05 14:20:24.012836,2019-04-08 07:40:31.940158,bbt98cb0cdd,33259.000,bbta8891916,34033,2018-02-05 14:20:24.012836,11:00:00,NaT,nan,97b6e7ec-8063-4511-b7b3-80b6c5cbdfe8
3,2008c3e2-07fb-4143-b1b9-267d4f83ea3c,00000000-0000-0000-0000-000000000001,24bf3729-46b3-47fa-ba19-6786f208758b,1,2,2,AT&T Wireless,2017-12-27 21:26:17.417336,2019-04-08 07:42:22.446417,bbtd87eaccd,33338.000,bbt8927d24c,29750,2017-12-27 21:26:17.417336,11:00:00,NaT,nan,97b6e7ec-8063-4511-b7b3-80b6c5cbdfe8
4,a7bdfa6e-ecdb-4993-8ae0-28edb365de66,00000000-0000-0000-0000-000000000001,d4d696af-37ff-427e-8035-2cb1bd630495,1,2,1,Verizon Wireless,2020-05-03 18:16:36.833859,2020-05-03 18:16:36.835001,bbtc8d25952,35532.000,bbtb7e0f69c,None,2020-05-03 18:16:36.833859,11:00:00,NaT,nan,00000000-0000-0000-0000-000000000003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122070,bb3d48cc-bc55-440a-b9f6-65257adb5b41,00000000-0000-0000-0000-000000000001,d4d696af-37ff-427e-8035-2cb1bd630495,1,2,2,Verizon Wireless,2018-01-30 02:53:31.266045,2019-04-08 06:58:43.321184,bbtcf0a991c,35536.000,bbt249aceb0,33334,2018-01-30 02:53:31.266045,11:00:00,NaT,nan,00000000-0000-0000-0000-000000000003
122071,bb434269-efc4-4181-8964-bc7691595e99,00000000-0000-0000-0000-000000000001,d4d696af-37ff-427e-8035-2cb1bd630495,1,2,5,"Metro PCS, Inc.",2017-06-19 12:41:23.893875,2019-04-08 06:56:21.460163,bbt724f21bf,35645.000,bbt76394567,9568,2016-07-07 11:29:00.000000,11:00:00,NaT,nan,00000000-0000-0000-0000-000000000003
122072,bb51bee3-5c0b-4268-8d8e-362e478fe8e8,00000000-0000-0000-0000-000000000001,d4d696af-37ff-427e-8035-2cb1bd630495,2,2,2,Cricket Wireless - ATT - SVR,2019-08-03 00:46:50.806718,2019-08-03 00:49:25.722288,bbt23b8ae10,38606.000,bbt5e16adea,None,2019-08-03 00:49:25.722288,11:00:00,NaT,nan,00000000-0000-0000-0000-000000000003
122073,c7184f87-34aa-4f9b-872c-9bf9ed5d2092,00000000-0000-0000-0000-000000000001,b98bbdf3-7a8b-4897-92cf-c32ed0b57606,1,3,2,Verizon Wireless,2020-07-16 17:30:31.614148,2020-07-18 18:01:18.479728,bbt969f2ea7,nan,bbt553a9d13,None,NaT,11:00:00,2020-07-18 18:01:18.479728,2.000,1f190fd1-386c-4baf-af1e-d436c09012de


In [ ]:
sql = '''
select s.id as subscriber_id,
sv.survey_id
from public.subscribers s
inner join public.surveys sv
on sv.id = r.survey_id
left join sg_types t
on sv.sg_type_id = t.id
left join sg_statuses ss
on sv.sg_type_id = ss.id
'''
surveys = pd.read_sql_query(sql, conn)
conn = None

In [119]:
sql = '''
select s.id as subscriber_id,
sv.title,
sv.created_at,
sv.updated_at,
r.submitted_at,
r.started_at,
r.response_time,
ss.name,
t.name
from public.subscribers s
left join public.sg_responses as r
on s.id = r.subscriber_id
left join public.surveys sv
on sv.id = r.survey_id
left join sg_types t
on sv.sg_type_id = t.id
left join sg_statuses ss
on sv.sg_type_id = ss.id
'''
surveys = pd.read_sql_query(sql, conn)
conn = None

In [120]:
surveys

,subscriber_id,title,created_at,updated_at,submitted_at,started_at,response_time,name,name
0,4f757ea8-ac8e-4872-9899-1b863f7a9f1f,Delta Dental Survey 2019,2019-01-27 22:48:28,2020-08-01 23:25:32.490943,2019-02-12 18:02:04,2019-02-12 18:01:54,10.000,Launched,Launched
1,213aa6b1-8a16-45b5-b6cf-01610942335b,Delta Dental Survey 2019,2019-01-27 22:48:28,2020-08-01 23:25:32.490943,2019-02-12 18:03:41,2019-02-12 18:02:21,80.000,Launched,Launched
2,92a15ce5-0a72-4a3d-8434-fa0a0f6a91b8,Delta Dental Survey 2019,2019-01-27 22:48:28,2020-08-01 23:25:32.490943,2019-02-12 18:03:47,2019-02-12 18:02:45,62.000,Launched,Launched
3,4e2ea4c7-589c-4cf6-9b8d-35a4e09dc2db,Delta Dental Survey 2019,2019-01-27 22:48:28,2020-08-01 23:25:32.490943,2019-02-12 18:03:56,2019-02-12 18:01:52,124.000,Launched,Launched
4,38f34551-869e-4cbb-af57-5b7b38d74eaf,Delta Dental Survey 2019,2019-01-27 22:48:28,2020-08-01 23:25:32.490943,2019-02-12 18:05:24,2019-02-12 18:05:05,19.000,Launched,Launched
...,...,...,...,...,...,...,...,...,...
133984,42773aff-e873-4a80-b7f4-ebaa8ffda4cd,None,NaT,NaT,NaT,NaT,nan,None,None
133985,bae533ef-9e16-40c6-b4cd-9e526ced8d9e,None,NaT,NaT,NaT,NaT,nan,None,None
133986,0353071e-d123-4db0-8c94-bdab7a596883,None,NaT,NaT,NaT,NaT,nan,None,None
133987,098ff42e-c367-4193-b0f1-92eee9cf7acf,None,NaT,NaT,NaT,NaT,nan,None,None


In [121]:
surveys['responded'] = np.where(pd.isnull(surveys['submitted_at']), 0, 1)

In [126]:
sent_surveys = surveys[surveys['title'].notnull()]

In [97]:
extrainfo[extrainfo['response_time'].isna()]

,id,sg_id,sg_survey_id,survey_id,subscriber_id,language_id,submitted_at,started_at,link_id,ip_address,response_time,longitude,latitude,country
335,71e29c73-2052-4da4-91d7-3726912eb7b7,339,4881773,9f8b0521-4d03-488e-8727-f70c548747c1,5a3ae26c-c30c-47e1-91e1-473e2dedf963,1,2019-06-28 14:27:47,2019-06-28 14:27:47,None,None,nan,None,None,None
336,00eee186-7dab-41f3-8efe-b4385df8a8a7,340,4881773,9f8b0521-4d03-488e-8727-f70c548747c1,98377445-7ccf-4e4f-8645-f9392fc2c568,1,2019-06-28 14:27:47,2019-06-28 14:27:47,None,None,nan,None,None,None
337,707f6bfa-651a-4d4c-a106-5ec86bf42d1b,341,4881773,9f8b0521-4d03-488e-8727-f70c548747c1,7f857df6-2ef0-4c74-ae21-78655e099f75,1,2019-06-28 14:27:47,2019-06-28 14:27:47,None,None,nan,None,None,None
338,59fb1569-c358-4696-87dc-7d9de5de2622,344,4881773,9f8b0521-4d03-488e-8727-f70c548747c1,20190022-170c-4e7b-98c8-97db47f2db11,1,2019-06-28 14:27:47,2019-06-28 14:27:47,None,None,nan,None,None,None
339,a6060183-6aa6-4909-a46e-03286f23dcd1,345,4881773,9f8b0521-4d03-488e-8727-f70c548747c1,9810eae1-21fd-45f4-9bc1-3b8f82bfd956,1,2019-06-28 14:27:47,2019-06-28 14:27:47,None,None,nan,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7325,3f34e3d8-33b3-45db-bfe0-6efbf3925a15,8862,4881773,9f8b0521-4d03-488e-8727-f70c548747c1,e3eff810-4b1b-4f4b-b2b1-7a22569078d3,2,2019-07-01 07:02:02,2019-07-01 07:02:02,None,None,nan,None,None,None
7326,df8af575-af8f-485a-a6cc-3976c45a5a9d,8863,4881773,9f8b0521-4d03-488e-8727-f70c548747c1,e210a776-00bd-4d97-9879-2f7184f1e7ce,2,2019-07-01 07:02:02,2019-07-01 07:02:02,None,None,nan,None,None,None
7327,9120e65a-6024-4991-a41a-5d355a7b4d45,8864,4881773,9f8b0521-4d03-488e-8727-f70c548747c1,dd7302ae-0047-4e00-8ccf-d69bd972631f,2,2019-07-01 07:02:02,2019-07-01 07:02:02,None,None,nan,None,None,None
16645,5b2f0aed-d19a-4cff-8908-3bcd47dee422,3,4534518,0704f40e-603b-49fb-bb72-9ef5785b2332,12969526-e029-475c-be96-ede6f3e12983,1,2019-03-31 17:24:00,2019-03-31 17:24:00,None,None,nan,None,None,None


In [134]:
# Load the messages dataset

messages = pd.read_csv(r'/Users/lisafan/Desktop/BBT/bbt_messages_2017_2020.csv', low_memory = False)

In [135]:
messages

,message_id,outbound_message_created_at,outbound_message_body,outbound_message_partner_id,message_type,subscriber_id,created_at,signedup_at,deactivated_at,carrier_name,subscriber_zip_code,subscriber_city,timezone_default_offset,timezone_name,subscriber_state,subscriber_source,subscriber_status,subscriber_language,subscriber_deactivation_method,partner_name,partner_created_date,partner_is_active,partner_state,children_count,dob_oldest_child,dob_youngest_child,scheduled_message_id
0,6e92ae11-f983-479f-8e9c-2b689368954d,2017-05-03 15:11:30.588685,Invalid zipcode. Kindly check your zipcode and...,00000000-0000-0000-0000-000000000002,signup,53e4abdd-308a-4cfc-a6fc-410c89a28d2d,2017-05-03 14:30:08.345377,2017-05-03 14:30:08.345377,NaN,AT&T Wireless,11096.000,Inwood,-5.000,America/New_York,New York,Text SignUp,activated,English,NaN,"WNET - New York, NY",2018-03-25 21:32:21.001262,True,New York,1.000,2015-06-06,2015-06-06,NaN
1,57c0ec0e-f812-4b45-a33a-213f4663f1c2,2017-05-09 16:34:15.603110,Is this zipcode correct? If yes reply with 'Y...,00000000-0000-0000-0000-000000000002,signup,f328ee46-398c-422f-aa55-4816caeae06e,2017-05-09 16:31:26.230154,2017-05-09 16:31:26.230154,NaN,"T-Mobile USA, Inc.",11219.000,Brooklyn,-5.000,America/New_York,New York,Text SignUp,activated,English,NaN,"WNET - New York, NY",2018-03-25 21:32:21.001262,True,New York,1.000,2014-06-23,2014-06-23,NaN
2,17d99d4d-137a-4317-b4d8-a63a4690fc93,2017-05-04 18:40:41.904659,"If you care for a young child, Bright by Text ...",f92494a8-bcce-473f-a56e-0ae2515c827e,signup,99915cfb-856b-4d59-8b94-0bd26d1342c4,2017-05-04 18:39:17.165132,2017-05-04 18:39:17.165132,NaN,"T-Mobile USA, Inc.",78251.000,San Antonio,-6.000,America/Chicago,Texas,Text SignUp,activated,English,NaN,KLRN - San Antonio,2018-08-28 20:58:04.700242,True,Texas,2.000,2014-09-02,2016-09-03,NaN
3,ffb5de23-d828-4e3a-becf-8b10b80cefba,2017-05-04 18:45:12.364195,Unrecognized date of birth format. Please res...,f92494a8-bcce-473f-a56e-0ae2515c827e,signup,99915cfb-856b-4d59-8b94-0bd26d1342c4,2017-05-04 18:39:17.165132,2017-05-04 18:39:17.165132,NaN,"T-Mobile USA, Inc.",78251.000,San Antonio,-6.000,America/Chicago,Texas,Text SignUp,activated,English,NaN,KLRN - San Antonio,2018-08-28 20:58:04.700242,True,Texas,2.000,2014-09-02,2016-09-03,NaN
4,bc56c5c2-c5b2-4a95-bfd5-a284661e7052,2017-05-04 18:39:17.856495,KLRN welcomes you to Bright by Text! Expect 2 ...,f92494a8-bcce-473f-a56e-0ae2515c827e,signup,99915cfb-856b-4d59-8b94-0bd26d1342c4,2017-05-04 18:39:17.165132,2017-05-04 18:39:17.165132,NaN,"T-Mobile USA, Inc.",78251.000,San Antonio,-6.000,America/Chicago,Texas,Text SignUp,activated,English,NaN,KLRN - San Antonio,2018-08-28 20:58:04.700242,True,Texas,2.000,2014-09-02,2016-09-03,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17349497,f6dfbbbd-1604-4ca8-b1c4-d75709a8641f,2020-04-02 22:05:25.285664,Are you worried about paying rent or utilities...,b6a20a01-a55e-481c-9878-7c0ce616374a,scheduled,ef99f698-0fce-46af-8a36-d6fb1e3019f3,2020-01-09 23:50:28.983049,2020-01-09 23:50:28.983049,NaN,US Cellular Corp.,53726.000,Madison,-6.000,America/Chicago,Wisconsin,Web SignUp,activated,English,NaN,Wisconsin Public Television,2019-04-17 19:52:45.009641,True,Wisconsin,1.000,2017-07-30,2017-07-30,14d3b7e6-0af2-4065-9168-72241a26e25e
17349498,f6dfbbbd-1604-4ca8-b1c4-d75709a8641f,2020-04-02 22:05:25.463028,Are you worried about paying rent or utilities...,b6a20a01-a55e-481c-9878-7c0ce616374a,scheduled,efa04d9a-bf9e-469f-825a-452fcc0e5754,2019-07-17 23:00:55.830226,2019-07-17 23:05:59.405609,NaN,Verizon Wireless,54311.000,Green Bay,-6.000,America/Chicago,Wisconsin,Text SignUp,activated,English,NaN,Wisconsin Public Television,2019-04-17 19:52:45.009641,True,Wisconsin,2.000,2014-10-18,2017-08-18,14d3b7e6-0af2-4065-9168-72241a26e25e
17349499,f6dfbbbd-1604-4ca8-b1c4-d75709a8641f,2020-04-02 22:05:26.491638,Are you worried about paying rent or utilities...,b6a20a01-a55e-481c-9